In [1]:
# switch to the project directory
%cd ..
# working directory should be ../pdi

c:\Users\admin\Desktop\research\pdi


In [2]:
import sys
import os
module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pdi.constants import (
    PARTICLES_DICT,
    TARGET_CODES
)
from pdi.data.constants import GROUP_ID_KEY

In [4]:
import torch
import torch.nn as nn
from pdi.data.preparation import FeatureSetPreparation
from pdi.models import AttentionModel
from pdi.data.types import Split
import onnx

device = torch.device("cpu")


data_preparation = FeatureSetPreparation()
(train_loader, ) = data_preparation.prepare_dataloaders(64, 0, [Split.TRAIN])

input_data, _, data_dict = next(iter(train_loader))
gid = data_dict.get(GROUP_ID_KEY)

dummy_input = (input_data.to(device), gid.to(device))

os.makedirs(f"onnx/Proposed/", exist_ok=True)
for target_code in TARGET_CODES:
    load_path = f"models/Proposed/{PARTICLES_DICT[target_code]}.pt"
    export_path = f"onnx/Proposed/{PARTICLES_DICT[target_code]}.onnx"
    saved_model = torch.load(load_path)
    model = AttentionModel(*saved_model["model_args"]).to(device)
    model.thres = saved_model["model_thres"]
    model.load_state_dict(saved_model["state_dict"])

    torch.onnx.export(model, dummy_input, export_path)

    onnx_model = onnx.load(export_path)
    onnx.checker.check_model(onnx_model)